In [41]:
import numpy as np
from numpy import linalg as LA
import math

In [42]:
x=np.array([[0,1],[2,3]])

In [43]:
X = np.array([[0, 2, 2], [0, 2, 1], [0, 2, 0],[1, 2, 0],[2, 2, 0],[0, 1, 2],[0, 1, 1],[0, 1, 0],[1, 1, 0],[2, 1, 0], [0, 0, 2],[0, 0, 1],[0, 0, 0],[1, 0, 0],[2, 0, 0]])

In [44]:
def normalizing_params(pts):
    centroid=np.mean(pts,axis=0)
    avg_distance=np.mean(LA.norm(pts-centroid, axis=1))
    return centroid,avg_distance


In [45]:
def normalizing_matrix(pts):
    c,d=normalizing_params(pts)
    n=pts.shape[1]
    d=d/math.sqrt(2)
    if n==2:
        mat=np.array([[1/d,0,-c[0]/d],[0,1/d,-c[1]/d],[0,0,1]])
    else:
        mat=np.array([[1/d,0,0,-c[0]/d],[0,1/d,0,-c[1]/d],[0,0,1/d,-c[2]/d],[0,0,0,1]])
    return mat

In [48]:
def homo_coord(pts):
    rows=pts.shape[0]
    pts=np.append(pts, np.ones((rows, 1)), axis=1)
    return pts

In [65]:
# zerocol=np.zeros((X.shape[0],4))
# print(zerocol)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [117]:
def Fiducial_matrix(X_pts,x_pts):
    rows=X_pts.shape[0]

    X_pts=homo_coord(X_pts)
    zerocol=np.zeros((rows,4))
    multx=(-X_pts.T*x_pts[:,0]).T
    multy=(-X_pts.T*x_pts[:,1]).T

    P=np.append(X_pts,zerocol,axis=1)
    P=np.append(P,multx,axis=1)
    P=np.append(P,zerocol,axis=1)
    P=np.append(P,X_pts,axis=1)
    P=np.append(P,multy,axis=1)
    
    P=np.reshape(P,(2*rows,12))
    return P

In [140]:
def P_estimate(X_pts,x_pts):
    P=Fiducial_matrix(X_pts,x_pts)
    u, s, vh = np.linalg.svd(P)
    # print(u)
    print(s)
    # print(vh)

In [141]:
X=np.array([[0,1,1],[1,2,3]])

In [142]:
P_estimate(X,x)

[14.61062038  4.18464506  1.51777664  0.84550106]


In [136]:
Fiducial_matrix(X,x)

array([[ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0., -0., -0., -0., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1., -0., -1., -1., -1.],
       [ 1.,  2.,  3.,  1.,  0.,  0.,  0.,  0., -2., -4., -6., -2.],
       [ 0.,  0.,  0.,  0.,  1.,  2.,  3.,  1., -3., -6., -9., -3.]])

In [46]:
normalizing_params(X)

(array([0.6, 1. , 0.6]), 1.3453274243577624)

In [47]:
normalizing_matrix(X)

array([[ 1.05120399,  0.        ,  0.        , -0.6307224 ],
       [ 0.        ,  1.05120399,  0.        , -1.05120399],
       [ 0.        ,  0.        ,  1.05120399, -0.6307224 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [50]:
homo_coord(X)

array([[0., 2., 2., 1.],
       [0., 2., 1., 1.],
       [0., 2., 0., 1.],
       [1., 2., 0., 1.],
       [2., 2., 0., 1.],
       [0., 1., 2., 1.],
       [0., 1., 1., 1.],
       [0., 1., 0., 1.],
       [1., 1., 0., 1.],
       [2., 1., 0., 1.],
       [0., 0., 2., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.],
       [2., 0., 0., 1.]])